In [14]:
import sqlite3
import os
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt



connexion = sqlite3.connect("data/bce.db")
cursor = connexion.cursor()

# Which percentage of the companies are under which juridical form?

### First step : GROUP BY Juridical Form

In [15]:
query1 = f"""
    SELECT 
        enterprise.JuridicalForm, count(*)
    FROM 
        enterprise
    WHERE 
        enterprise.JuridicalForm IS NOT NULL
    GROUP BY
        enterprise.JuridicalForm

    LIMIT 10;   
"""        

cursor.execute(query1)
cursor.fetchall()

[('001', 7),
 ('002', 160),
 ('003', 7673),
 ('006', 4648),
 ('007', 2),
 ('008', 12487),
 ('009', 18),
 ('011', 23453),
 ('012', 33097),
 ('013', 1621)]

### Second step : Find the SUM of all enterprises

In [16]:
query2 = f"""
    SELECT 
        count(enterprise.JuridicalForm)
    FROM 
        enterprise

"""        

cursor.execute(query2)
cursor.fetchall()

[(1099649,)]

### Third Step : Percentage

In [17]:
query3 = f"""
    SELECT
        enterprise.JuridicalForm,
        COUNT(enterprise.JuridicalForm) as 'Count by juridical form',
        COUNT(enterprise.JuridicalForm) / (SELECT COUNT(enterprise.JuridicalForm) FROM enterprise) * 100.0 as 'Percentage'
    FROM 
        enterprise
    WHERE 
        enterprise.JuridicalForm IS NOT NULL
    GROUP BY
        enterprise.JuridicalForm
    ;   
"""        

cursor.execute(query3)
cursor.fetchall()

[('001', 7, 0.0),
 ('002', 160, 0.0),
 ('003', 7673, 0.0),
 ('006', 4648, 0.0),
 ('007', 2, 0.0),
 ('008', 12487, 0.0),
 ('009', 18, 0.0),
 ('011', 23453, 0.0),
 ('012', 33097, 0.0),
 ('013', 1621, 0.0),
 ('014', 99035, 0.0),
 ('015', 356509, 0.0),
 ('016', 11655, 0.0),
 ('017', 140375, 0.0),
 ('018', 376, 0.0),
 ('019', 1699, 0.0),
 ('020', 1633, 0.0),
 ('021', 79, 0.0),
 ('022', 852, 0.0),
 ('023', 33, 0.0),
 ('025', 2156, 0.0),
 ('026', 1940, 0.0),
 ('027', 23, 0.0),
 ('028', 2, 0.0),
 ('029', 442, 0.0),
 ('030', 61875, 0.0),
 ('040', 5, 0.0),
 ('051', 234, 0.0),
 ('060', 243, 0.0),
 ('065', 285, 0.0),
 ('070', 94015, 0.0),
 ('108', 34, 0.0),
 ('110', 5, 0.0),
 ('114', 52, 0.0),
 ('116', 39, 0.0),
 ('117', 27, 0.0),
 ('123', 24, 0.0),
 ('124', 5084, 0.0),
 ('125', 2380, 0.0),
 ('127', 1, 0.0),
 ('129', 333, 0.0),
 ('151', 267, 0.0),
 ('160', 183, 0.0),
 ('200', 3, 0.0),
 ('217', 7, 0.0),
 ('218', 5, 0.0),
 ('230', 4005, 0.0),
 ('235', 12321, 0.0),
 ('265', 7, 0.0),
 ('301', 12, 0.0)

### Extract JuridicalForm Code and French Description string

In [18]:
query4 = f"""
    SELECT
        code.Code as 'Code',
        code.Description as 'Description'
    FROM 
        code
    WHERE
        code.Category is "JuridicalForm" AND
        code.Language is "FR"
        ;
     """
     
cursor.execute(query4)
cursor.fetchall()



[('001', 'Société coopérative européenne'),
 ('002', 'Organisme de financement de pensions'),
 ('003', 'Unité TVA'),
 ('006', 'Société coopérative à responsabilité illimitée'),
 ('007',
  'Société coopérative à responsabilité illimitée et solidaire, société coopérative de participation'),
 ('008', 'Société coopérative à responsabilité limitée'),
 ('009',
  'Société coopérative à responsabilité limitée, coopérative de participation'),
 ('010', 'Société privée à responsabilité limitée unipersonnelle'),
 ('011', 'Société en nom collectif'),
 ('012', 'Société en commandite simple'),
 ('013', 'Société en commandite par actions'),
 ('014', 'Société anonyme'),
 ('015', 'Société privée à responsabilité limitée'),
 ('016', 'Société coopérative (ancien statut)'),
 ('017', 'Association sans but lucratif'),
 ('018', "Etablissement d'utilité publique"),
 ('019', 'Mutualité / Société Mutualiste / Union nationale de mutualités'),
 ('020', 'Union professionnelle'),
 ('021',
  "Association d'assurances

### Replace JuridicalForm code by the french Description of the Code

In [19]:
# SELECT 
# FROM 
# INNER JOIN 
# WHERE 
# GROUP BY

query6 = f"""
    SELECT
        enterprise.JuridicalForm as 'CodeOfJuridicalForm',
        code.Description as 'CodeDescription',
        COUNT(*) as 'CountByGroupBy'
          
    FROM
        enterprise
                        
    INNER JOIN
        code ON enterprise.JuridicalForm = code.Code

    WHERE
        enterprise.JuridicalForm IS NOT NULL AND
        code.Category = 'JuridicalForm' AND
        code.Language = 'FR'        
        
    GROUP BY
        enterprise.JuridicalForm
        
    ORDER BY
        CountByGroupBy DESC
                
    LIMIT 40
    ;  
        
        """
        
cursor.execute(query6)
cursor.fetchall()

[('015', 'Société privée à responsabilité limitée', 356509),
 ('610', 'Société à responsabilité limitée', 170607),
 ('017', 'Association sans but lucratif', 140375),
 ('014', 'Société anonyme', 99035),
 ('070', 'Association des copropriétaires', 94015),
 ('030', 'Entité étrangère', 61875),
 ('012', 'Société en commandite simple', 33097),
 ('721', 'Société ou association sans personnalité juridique', 25178),
 ('011', 'Société en nom collectif', 23453),
 ('612', 'Société en commandite', 17228),
 ('008', 'Société coopérative à responsabilité limitée', 12487),
 ('235',
  'Entité étrangère sans établissement belge avec représentant responsable TVA',
  12321),
 ('016', 'Société coopérative (ancien statut)', 11655),
 ('003', 'Unité TVA', 7673),
 ('124', 'Etablissement public', 5084),
 ('006', 'Société coopérative à responsabilité illimitée', 4648),
 ('230',
  'Entité étrangère avec un bien immobilier en Belgique (avec personnalité juridique)',
  4005),
 ('125', 'Association internationale san

### Define the DataFrame

In [20]:
df_juridicalform = pd.read_sql_query(query6, connexion)

### First Pie Chart

In [21]:
# st.write("Here's our first attempt at using data to create a table:")
# st.write(pd.DataFrame(df))

labels = df_juridicalform['CodeDescription']
sizes = df_juridicalform['CountByGroupBy']

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax1.axis('equal')
ax1.set_title("Question 1 : Which percentage of the companies are under which juridical form?")

st.pyplot(fig1)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

# Which percentage of the companies are under which Status?

In [22]:
# SELECT 
# FROM 
# INNER JOIN 
# WHERE 
# GROUP BY

query7 = f"""
    SELECT
        enterprise.Status as 'Status',
        code.Description as 'CodeDescription',
        COUNT(*) as 'CountByGroupBy'
          
    FROM
        enterprise
                        
    INNER JOIN
        code ON enterprise.Status = code.Code

    WHERE
        enterprise.Status IS NOT NULL AND
        code.Category = 'Status' AND
        code.Language = 'FR'        
        
    GROUP BY
        enterprise.Status
        
    ORDER BY
        CountByGroupBy DESC
                
    LIMIT 40
    ;  
        
        """
        
cursor.execute(query7)
cursor.fetchall()

[('AC', 'Actif', 1863292)]

In [23]:
df_status = pd.read_sql_query(query7, connexion)

labels = df_status['CodeDescription']
sizes = df_status['CountByGroupBy']

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax1.axis('equal')

ax1.set_title("Question 2 : Which percentage of the companies are under which Status?")

st.pyplot(fig1)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

# Which percentage of the companies are which type of entreprise?

In [24]:
# SELECT 
# FROM 
# INNER JOIN 
# WHERE 
# GROUP BY

query8 = f"""
    SELECT
        enterprise.TypeOfEnterprise as 'TypeOfEnterprise',
        code.Description as 'CodeDescription',
        COUNT(*) as 'CountByGroupBy'
          
    FROM
        enterprise
                        
    INNER JOIN
        code ON enterprise.TypeOfEnterprise = code.Code

    WHERE
        code.Category = 'TypeOfEnterprise' AND
        code.Language = 'FR'        
        
    GROUP BY
        enterprise.TypeOfEnterprise
        
    ORDER BY
        CountByGroupBy DESC
                
    LIMIT 40
    ;  
        
        """
        
cursor.execute(query8)
cursor.fetchall()

[('2', 'Personne morale', 1099649), ('1', 'Personne physique', 763643)]

In [25]:
df_typeof = pd.read_sql_query(query8, connexion)

labels = df_typeof['CodeDescription']
sizes = df_typeof['CountByGroupBy']

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax1.axis('equal')
ax1.set_title("Question 3 : Which percentage of the companies are which type of entreprise?")


st.pyplot(fig1)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

# What is the average company's age in each sector (hint: look what is the NACE code)?

In [34]:
import sqlite3
import os
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt



connexion = sqlite3.connect("data/bce.db")
cursor = connexion.cursor()

# SELECT 
# FROM 
# INNER JOIN 
# WHERE 
# GROUP BY

# select code.Code, code.Category, code.Description  from code where code.Category = "Nace2008"
# select activity.EntityNumber, activity.NaceCode from activity
# select enterprise.Number, enterprise.StartDate from enterprise

# STRFTIME('%d/%m/%Y, %H:%M', sale_datetime)


query9 = f"""
    SELECT
        enterprise.EnterpriseNumber as 'EnterpriseNumber',
        SUBSTR(activity.NaceCode, 1, 2) as 'NaceCode',
        code.Description as 'DescriptionCode',
        ROUND(AVG(STRFTIME('%Y', enterprise.StartDate))) as 'AverageStartDate',
        COUNT(*) as 'CountByGroupBy'
          
    FROM
        enterprise
                        
    INNER JOIN
        code ON NaceCode = code.Code,
        activity ON enterprise.EnterpriseNumber = activity.EntityNumber

    WHERE
        code.Category = 'Nace2008' AND
        code.Language = 'FR' AND
        activity.NaceVersion = '2008'   

    GROUP BY
        SUBSTR(activity.NaceCode, 1, 2)
                
    ORDER BY
        CountByGroupBy DESC
    ;  
        
        """
        
cursor.execute(query9)
cursor.fetchall()

df_NACE = pd.read_sql_query(query9, connexion)

In [37]:
df_NACE = df_NACE.drop(['EnterpriseNumber'], axis=1)
df_NACE['AverageStartDate'] = df_NACE['AverageStartDate'].astype(int)
df_NACE

,NaceCode,DescriptionCode,AverageStartDate,CountByGroupBy
0,43,Autres travaux de finition,2012,381364
1,47,Commerce de détail d'armes et de munitions en ...,2009,334782
2,70,Conseil pour les affaires et autres conseils d...,2013,165054
3,46,Commerce de gros de matériel agricole,2006,160108
4,82,Autres activités de soutien aux entreprises n....,2014,129789
...,...,...,...,...
83,12,Fabrication de produits à base de tabac,1992,53
84,98,Activités indifférenciées des ménages en tant ...,2011,14
85,07,Extraction de minerais de fer,2004,11
86,06,Extraction de gaz naturel,2015,8


In [ ]:
codes = df_NACE['NaceCode']
dates = df_NACE['AverageStartDate']

fig1, ax1 = plt.subplots()
ax1.bar_chart(sizes, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax1.axis('equal')
ax1.set_title("Question 4 : What is the average company's age in each sector (hint: look what is the NACE code) ?")


st.pyplot(fig1)